# Projet : Partie 2/2

Considérons le problème elliptique suivant : nous cherchons une fonction $u:[0,1]\to\mathbb{R}$ qui vérifie le problème
\begin{equation}
\left\lbrace
\begin{aligned}
-(a(x)*u'(x))' = f(x),&&0<x<1,\\
u(0)=u(1)=0
\end{aligned}
\right.
\end{equation}




où $f:[0,1]\to\mathbb{R}$ est une fonction régulière (par exemple continue) et :
$$a(x)=\begin{equation}
\left\lbrace
\begin{aligned}
a_L, &&0<x<x_0,\\
a_R, &&x_0<x<1,
\end{aligned}
\right.
\end{equation}$$

Résoudre ce problème est equivalent à résoudre les deux problèmes suivant :
\begin{equation}
\left\lbrace
\begin{aligned}
-a_L * u_1''(x) = f(x),&&0<x<x_0,\\
u(0)=0,  \\
u(x_0) =u_0
\end{aligned}
\right.
\end{equation}
et
\begin{equation}
\left\lbrace
\begin{aligned}
-a_R * u_2''(x) = f(x),&&x_0<x<1,\\
u(x_0)=u_0,  \\
u(1) =0
\end{aligned}
\right.
\end{equation}
tel que :$$a_R*u_2'(x_0^+)=a_L*u_1'(x_0^-)$$



In [1]:
#include <iostream>
#include <vector>
#include <stdexcept>

#include "xplot/xfigure.hpp"
#include "xplot/xmarks.hpp"
#include "xplot/xaxes.hpp"

#include "mesh1D.hpp"
#include "solution1D.hpp"
#include "matrix.hpp"
#include "lu.hpp"

### 1. 

Créez un objet de type `parameters` contenant les informations suivantes
* `xmin` l'abscisse du bord gauche ($0$ dans notre cas)
* `xmax` l'abscisse du bord droit ($1$ dans notre cas)
* `Nx` le nombre de points du maillage (à votre convenance mais ne soyez pas trop gourmand dans un premier temps)
* `node-centered` qui doit prendre la valeur $1$ de manière à construire un maillage centré sur les noeuds (et pas sur les cellules)

### 2. 

Créez une `lambda`-fonction pour le second membre de l'équation. Ici nous pourrons commencer avec la fonction $f(x) = 1$, $\forall x\in[0,1]$.

In [2]:
double max(std::vector<double> x)
{
    double maximum =x[0];
    for(int i = 1; i<x.size();i++)
    {
        if(x[i]>maximum)
        {maximum = x[i];}
    }
    
    return maximum;
}





In [3]:
double max_ab(double a, double b)
{
    if(a>b)
    {return a;}
    else {return b;}
}

In [4]:
double aL = (rand()/(double)RAND_MAX ) * (5.0-0) + 0; // Nmbre aléa entre 0 et 2
double aR = (rand()/(double)RAND_MAX ) * (1.5-0) + 0 ;
double x0 = (rand()/(double)RAND_MAX ) * (0.7-0.3) + 0.3;
double ld = 0;

In [5]:
/*On proportionne notre maillage pour que le pas de temps sur l'intervalle [0,x0] soit 
proche de celui de l'intervalle [x0,1]*/

int N3=256;
int Nx1=int(N3*x0)+1;
int Nx2=N3-Nx1+1;

parameters p = {
    {"xmin",0.},
    {"xmax",x0},
    {"Nx",Nx1},
    {"cell-centered",1}
};

parameters p2 = {
    {"xmin",x0},
    {"xmax",1.},
    {"Nx",Nx2},
    {"cell-centered",1}
};


In [6]:
double f(double x ) {return 1.;}

On prend f = 1, d'où :
------------
------------

\begin{equation}
u_1(x) = -\frac{x²}{2a_L} + x(\frac{λ}{x_0}+\frac{x_0}{2a_L})  
\end{equation}

--------------------------------
-------------------------------


\begin{equation}
u_2(x) = -\frac{x²}{2a_R} + x(\frac{λ}{x_0-1} - \frac{x_0 + 1}{2a_R}) - \frac{x_0}{2a_R} - \frac{λ}{x_0 -1}
\end{equation}

---------------------
---------------------

In [7]:
double u0=(x0-1)/(2*(aL-aR-(aL/x0)));

In [8]:
// Solution pour u1

std::vector<double> sol(std::vector<double> x)  
{
    std::vector<double> y(x.size());
    for(int k = 0; k < x.size(); k++){
        y[k] = -(x[k]*x[k])*(0.5/aL) + x[k]*(u0/x0 + 0.5*x0/aL);
    }
    return y;
}



In [9]:
// Solution pour u2
std::vector<double> sol2(std::vector<double> x)  
{
    std::vector<double> y(x.size());
    for(int k = 0; k < x.size(); k++){
        y[k] = -(x[k]*x[k])*(0.5/aR) + x[k]*(u0/(x0 -1) + 0.5*(x0+1)/aR) - 0.5*x0/aR - u0/(x0-1);
    }
    return y;
}

###  4.  

* Nous fabriquons tous les éléments du problème : 
    * les solutions $S_1$ et $S_2$ (objet de type `solution1D`) à partir des paramètres,
    * la matrice $A$ du Laplacien dont la taille est $N{\times}N$ avec $N=N_x-2$ où $N_x$ est le nombre de points du maillage comprenant les deux points de bord, 
    * le second membre $F$ qui doit être un `vector` de taille $N$ également ;
* Nous résolvons le système linéaire en utilisant la décomposition $LU$ de la matrice $A$. Le vecteur `u_in` obtenu est un vecteur de taille $N$ ;
* copiez les valeurs de `u_in` à leur place dans le vecteur `S.u` qui est de taille $N_x=N+2$.

In [10]:
solution1D S(p);
solution1D S2(p2);

auto N = S.m.Nx - 2 ;
auto N1 = S.m.Nx - 2 ;
auto N2 = S2.m.Nx - 2 ;
auto Nx = S.m.Nx+S2.m.Nx-1;
auto invh = (N1+1)/x0;
matrix A11 = 2 * identity(N1,N1) - identity(N1,N1,1) - identity(N1,N1,-1);

matrix A1 = aL * invh*invh* A11;

std::vector<double> F(N1);
std::transform(std::next(S.m.x.begin()),std::prev(S.m.x.end()),F.begin(),f);
F[N1-1] += aL*ld*invh*invh;

matrix LU1 = lu(A1);

std::vector<double> sol_n = solve_lu(LU1,F);
for(int k = 1 ; k<S.u.size() - 1;++k)
    S.u[k] = sol_n[k-1];

S.u[N1+1] = ld;


std::vector<double> F2(N2);

auto invh2 = (N2+1)/(1-x0);
std::transform(std::next(S2.m.x.begin()),std::prev(S2.m.x.end()),F2.begin(),f);
F2[0] += aR*ld*invh2*invh2;


matrix A22 = 2 * identity(N2,N2) - identity(N2,N2,1) - identity(N2,N2,-1);
matrix A2 = A22*aR*invh2*invh2;
matrix LU2 = lu(A2);
std::vector<double> sol2_n = solve_lu(LU2,F2);
for(int k = 1 ; k<S2.u.size() - 1;++k)
    S2.u[k] = sol2_n[k-1];
S2.u[0] = ld;


# ici nous résolvons numeriquement les équations suivantes
\begin{equation}
\left\lbrace
\begin{aligned}
-a_L * S_1''(x) = f(x),&&0<x<x_0,\\
S_1(0)=0,  \\
S_1(x_0) =0
\end{aligned}
\right.
\end{equation}
et
\begin{equation}
\left\lbrace
\begin{aligned}
-a_R * S_2''(x) = f(x),&&x_0<x<1,\\
S_2(x_0)=0,  \\
S_2(1) =0
\end{aligned}
\right.
\end{equation}

In [11]:
xpl::linear_scale sx, sy, sx2,sy2;
sx.min = p["xmin"], sx.max = p2["xmax"];
sx2.min = p["xmin"], sx2.max = p2["xmax"];
sy.min = 0., sy.max = max_ab(max(S.u),max(S2.u));

// fine mesh for the exact solution
auto NN = 1025;
double dx = x0/(NN-1);
std::vector<double> xx(NN);
for (auto i=0; i<NN; ++i)
    xx[i] = i*dx;

double dx2 = (1-x0)/(NN-1);
std::vector<double> xx2(NN);
for (auto i=0; i<NN; ++i)
    xx2[i] = x0+i*dx2;

auto ax_x = xpl::axis_generator(sx)
    .label("x")
    .finalize();
auto ax_y = xpl::axis_generator(sy)
    .orientation("vertical")
    .side("left")
    .finalize();

auto line = xpl::lines_generator(sx, sy)
    .x(xx)
    .y(sol(xx))
    .colors(std::vector<std::string>({"red"}))
    .labels(std::vector<std::string>({"exact"}))
    .display_legend(true)
    .finalize();

auto line2 = xpl::lines_generator(sx2, sy)
    .x(xx2)
    .y(sol2(xx2))
    .colors(std::vector<std::string>({"orange"}))
    .labels(std::vector<std::string>({"exact 2"}))
    .display_legend(true)
    .finalize();


auto scatter = xpl::scatter_generator(sx, sy)
    .x(S.m.x)
    .y(S.u)
    .colors(std::vector<xtl::xoptional<std::string>>{"navy"})
    .labels(std::vector<std::string>({"S1"}))
    .display_legend(true)
    .marker("circle")
    .finalize();

auto scatter2 = xpl::scatter_generator(sx2, sy)
    .x(S2.m.x)
    .y(S2.u)
    .colors(std::vector<xtl::xoptional<std::string>>{"green"})
    .labels(std::vector<std::string>({"S2"}))
    .display_legend(true)
    .marker("circle")
    .finalize();

auto fig = xpl::figure_generator()
    .padding_x(0.025)
    .padding_y(0.1)
    .title("solution with N = " + std::to_string(S.m.Nx+S2.m.Nx-1))
    .legend_location("top-left")
    .finalize();


std::cout<<aL*(S.u[S.u.size()-2]-S.u[S.u.size()-1])/dx;



fig.add_mark(scatter);
fig.add_mark(scatter2);
fig.add_mark(line);
fig.add_mark(line2);
fig.add_axis(ax_x);
fig.add_axis(ax_y);
fig.display();

// ---------------------- //


A Jupyter widget

1.98397

ici $S_1$ et $S_2$ sont solutions réspectivement de :
$$
\begin{equation}
\left\lbrace
\begin{aligned}
-a_L * S_1''(x) = f(x),&&0<x<x_0,\\
S_1(0)=0,  \\
S_1(x_0)= 0 \\
\end{aligned}
\right.
\end{equation}
$$
et
$$
\begin{equation}
\left\lbrace
\begin{aligned}
-a_R * S_2''(x) = f(x),&&x_0<x<1,\\
S_2(x_0)=0, \\
S_2(1)=0  \\
\end{aligned}
\right.
\end{equation}
$$

In [12]:
std::cout<< "x0 = " << x0 << std::endl;
std::cout<< "aL = " << aL << std::endl;
std::cout<< "aR = " << aR << std::endl;

x0 = 0.453401
aL = 3.39648
aR = 1.40204


### Estimation de u0

On a trouver que : $$ aR*u'(x_0^+) =aL*u'(x_0^-) .$$

De plus on sait que : $$ u'(x_0^+)=S_2'(x_0^+)- \frac{u_0}{1-x_0} $$
et : $$ u'(x_0^-)=S_1'(x_0^-)+ \frac{u_0}{x_0} $$
on en déduit que :
$$ u_0=\frac {a_R S_2'(x_0^+)-a_L S_1'(x_0^-)}{\frac{a_L}{x_0}+\frac{a_R}{1-x_0}} $$
on estime $S_1'(x_0^-)$ et $S_2'(x_0^+)$ de la maniere suivante :
$$S_1'(x_0^-)=\frac {S_1(x_0)-S_1(x_0-\Delta_x)}{\Delta_x}$$
$$S_2'(x_0^+)=\frac {S_2(x_0+\Delta_x)-S_2(x_0)}{\Delta_x}$$


In [13]:
//estimation de u0
double S1P=(S.u[S.u.size()-1]-S.u[S.u.size()-2])*invh;
double S2P=(S2.u[1]-S2.u[0])*invh2;
double u02=(aR*S2P-aL*S1P)/(aL/x0+aR/(1-x0));

//On peut constater la bonne estimation de u0
std::cout<<u0<<"//"<<u02<<std::endl;

0.0497208//0.049331


on cherche $u_1$ et $u_2$ solutions de :
$$
\begin{equation}
\left\lbrace
\begin{aligned}
-a_L * u_1''(x) = f(x),&&0<x<x_0,\\
u_1(0)=0,  \\
u_1(x_0)=u_0 \\
\end{aligned}
\right.
\end{equation}
$$
et
$$
\begin{equation}
\left\lbrace
\begin{aligned}
-a_R * u_2''(x) = f(x),&&x_0<x<1,\\
u_2(x_0)=u_0 \\
u_2(1)=0,  \\
\end{aligned}
\right.
\end{equation}
$$

In [14]:
//retranchement (on ajoute les fonctions affines a S1 et S2) des solutions avec u0=0 aux solutions exactes avec u0=u(x0)

for(int k = 0 ; k<S.u.size(); ++k){
    S.u[k] = S.u[k]+u02/x0*x0*k/S.u.size(); 
}
for(double k = 0; k<S2.u.size(); ++k){
    S2.u[k] = S2.u[k]+(1-x0-(k+1)/S2.u.size()*(1-x0))/(1-x0)*u02;
}


In [15]:

sx.min = p["xmin"], sx.max = p2["xmax"];
sx2.min = p["xmin"], sx2.max = p2["xmax"];
sy.min = 0., sy.max = max_ab(max(S.u),max(S2.u));


auto scatter3 = xpl::scatter_generator(sx, sy)
    .x(S.m.x)
    .y(S.u)
    .colors(std::vector<xtl::xoptional<std::string>>{"navy"})
    .labels(std::vector<std::string>({"numeric"}))
    .display_legend(true)
    .marker("circle")
    .finalize();

auto scatter4 = xpl::scatter_generator(sx2, sy)
    .x(S2.m.x)
    .y(S2.u)
    .colors(std::vector<xtl::xoptional<std::string>>{"green"})
    .labels(std::vector<std::string>({"numeric 2"}))
    .display_legend(true)
    .marker("circle")
    .finalize();

auto fig2 = xpl::figure_generator()
    .padding_x(0.025)
    .padding_y(0.1)
    .title("solution with N = " + std::to_string(S.m.Nx+S2.m.Nx-1))
    .legend_location("top-left")
    .finalize();



std::cout<<aL*(S.u[S.u.size()-2]-S.u[S.u.size()-1])/dx;



fig2.add_mark(scatter3);
fig2.add_mark(scatter4);
fig2.add_mark(line);
fig2.add_mark(line2);
fig2.add_axis(ax_x);
fig2.add_axis(ax_y);
fig2.display();

// ---------------------- //


A Jupyter widget

-1.25034